# Wrangle OpenStreetMap Data
## Cambridge, MA

I live very close to Cambridge. I go to Cambridge Public Library nearly every week to play go. Therefore, I am interested in finding out a bit more about the town. I have chosen to assess OpenStreetMap's data around Cambridge. To be more precise, I will be analyzing the data in the box created by the following lattitudes and longitudes.

```
MIN_LAT = 42.3409
MAX_LAT = 42.4162
MIN_LON = -71.1995
MAX_LON = -71.0251
```

In order to find out more about one of my favorite areas in the world, I will not only need to get the data, but also perform some housekeeping to clean and organize the data. If possible, I also want to see if I can help improve Open Street Map project's data about Cambridge. I will start by seeing what kind of data I am getting, then I will investigate what's the best way to programmatically improve the data.

# Interesting Encounters in the Data

## Nodes Not in the Box

This was a little unexpected. As a sanity check, I tested to make sure that all nodes laid within the box of interest. To my surprise, a significant number of the nodes appeared to be outside of the box that I defined.

There were 769,476 nodes in my dataset. 14,135 nodes were outside of the box. It's less than 2% of the total number of nodes, but this is definitely higher than I expected. However, these nodes are easily filtered out.

## 0 - 11 Year Old Data

While map data might not change very quickly, it does change. A business might have opening a new location, or closed down one of its store fronts. Because I am in Cambridge fairly often, I know that Toscanini's, which offers the best ice cream in Boston, has moved to a temporary location as its building is being renovated. However, this information might not show up if the information related to Tuscanini's is more than 6 months old.

There is a not insignificant range for when the data was last captured. Most of the information seems to have been gather between 2009 - 2013.

```
2007: 6070
2008: 1846
2009: 443962
2010: 43823
2011: 40810
2012: 45535
2013: 163541
2014: 29466
2015: 23821
2016: 49911
2017: 29060
2018: 20020
```

## Bewildering Organization of Data

The organization of the data seems haphazard in "node_tags" and "way_tags". There does not seem to exist a standard way to input description of a feature on the node or way into the node and way tags system. Beyond "id" field, which is a numeric id, value, key and type information seemed to be mixed together. Value information seems to vary especially broadly. It ranges from telephone numbers, to an URL, and to a short paragraph description.

## Meet the Maker

It is perhaps expected from Pareto Principle that vast majority of the edits are contributed by a few users. I would hazard a guess that many of the major contributors also used an automated way to generate their edits. Subsequently, people who have a special interest in a particular location filled in more details or correct whatever was wrong with the previous data point.

Another unexpected discovery that came out of this was that most node nodes were created once and never updated. While way nodes were updated more often. On average, a way node was created and then just update once. This makes me question how up to date the information on this patch of Open Street Maps are, when we combine this discover with the discovery that most of the information was created in 2009-2013. 

User Node Creators:

```
Top 10 out of 999:

crschmidt -- 432,048
jremillard-massgis -- 50,764
OceanVortex -- 37,770
morganwahl -- 27,495
ryebread -- 25,199
ingalls_imports -- 18,497
mapper999 -- 11,238
cspanring -- 5,826
JasonWoof -- 5,548

277/999 have only made one edit.

Total Node Edits: 957,679 for 755,341 node nodes
```

Way Node Creators:

```
Top 10 out of 771:

jremillard-massgis -- 82,295
ryebread -- 6,899
wambag -- 3,556
OceanVortex -- 3,161
ingalls_imports -- 2,794
mapper999 -- 2,022
morganwahl -- 1,509
MassGIS Import -- 1,351
cspanring -- 1,252
cfogel -- 1,193

234/771 have one made one edit.

Total Way Edits: 267,373 for 128,389 way nodes
```

# Scope of This Project

## Fixing Addresses

With the above information, it seems the most valuable piece that I can contribute programmatically would be to edit the semi-permanent information, namely street addresses, since street name changes much less frequently than a business store front.

One important means of standardization is to make sure all street names use the same convention. Cambridge is one of the oldest part of the United States. There are more unusual naming conventions. In addition to trying to convert a diverse range of abbreviations (St, st, ST, and St. for Street), there are a wide range of acceptable street name ending conventions. Here is a sample:

```
"Street"
"Avenue"
"Boulevard"
"Drive"
"Court"
"Place"
"Square"
"Lane"
"Road"
"Trail"
"Parkway"
"Commons"
"Broadway"
"Terrace"
"Highway"
"Way"
"Circle"
"Driveway"
"Plaza"
"Park"
"Mall"
"Fenway"
"Row"
```

I will convert abbreviations into the fully expanded term. It will make it easier for those to work with the database afterwards to choose their own abbreviates as policy rather than have it as a part of the mechanism.

The address information is usually located within way tags and node tags "value" fields with "addr" as its type. In addition to street address, zip code, city, state, and even country information are also sometimes located in it. I will eliminate abbreviations used for city and state as well as for street addresses. I will shorten all zip code into its five digit format.

I will use a simple substitution method for making the corrections. I will create a dictionary with words that I am looking to change as keys, and with the words to be changed to as the value. Whenever I encounter a key, I will replace it to the value.

# Problems Encounter and Potential Solutions

## Poor Organization of Information

The "addr" tag is associated with a wild variety of types of data. The associated value could be a zip code, a town name, a street number, or a combination of the above and more. As the result some associate values are overloaded with information, while others are underutilized.

## Relying on Heuristic

In order to minimize the amount of data that need to be hand coded, I relied on heuristics to narrow down problematic values that needed my individual attention. This means that I could be making type II errors.

## Missing a Complete Picture

In essence, I am trying to correct data on the level of individual words. The organization of the tag information already caused a fragmentation of information regarding the associated node. I am looking for correctness at the individual word level. Higher level syntax and semantics had to be ignored.

# Overview of the Data

## Files

```
cambridge.xml -- 171 MB
nodes.csv ------  60 MB
nodes_tags.csv -   4 MB
ways.csv -------   8 MB
ways_nodes.csv -  22 MB
ways_tags.csv --  10 MB
cambridge.db ---  90 MB
```

## Data

```
Nodes -- 755,341
Ways  -- 128,389
```

# Investigating the Data

## Cameras and Surveillance

Privacy is becoming a more and more prevalent concern. When I encountered 'camera' type in nodes tags, I became immediately interested. I was curious to see how common these cameras were.

```SQL
SELECT COUNT(*) FROM nodes_tags WHERE type LIKE "%camera%"
```

```SQL
236
```

```SQL
SELECT COUNT(*) FROM nodes_tags WHERE type LIKE "%surveillance%"
```

```SQL
207
```

Luckily, it does not seem to be widespread yet, at the time when the data was captured. However, I also must wonder if there might not be many more cameras that are more well hidden.

## Parking

Parking in Cambridge can be very expensive. The daily parking rate for parking garages in the busiest parts of the city can cost $40+. Therefore, I am always on the lookout for good parking spots.

```SQL
SELECT COUNT(*) FROM ways_tags WHERE type LIKE "%parking%";
```

```SQL
128
```

Compared with the number of nodes for the city, the records for street parking is surprising few. Perhaps, this is why I, and probably many others must constantly try to find good parking spaces.

# Ideas for Future

One limitation of my approach is that the corrections is limited to individual word level. In order to programmatically make semantic level corrections, I would need some way to reading all ways tags for a ways node, make sense of the information therein, and restructure the information in such a way that it makes sense for the next person or program that acquires the data. At this moment, I do not have a good way to do that. However, I think that would be a good goal to work towards.

One potential improvement on my current procedure might be to expand the looping structure for how I approached data cleaning. The approach that I took repeatedly went through the XML data. Between each time, I would make adjustments to further improve the program logic. This limited me to making adjustment on the individual word level.

If I had more time, I should integrate data exploration phase with data cleaning phase more tightly. Looking through the data during exploration, I noticed places where, even only making adjustment at word level, I could have improved the overall structure of the data.

```SQL
SELECT COUNT(*) FROM ways_tags WHERE type LIKE "%addr2%";
```

```SQL
1
```

It wasn't until I looked at the data in the database, I noticed this very strange type. It would have been easily correctly during the data cleaning phase, if I had known. However, I could not have known until I had some preliminary results to work with.

Finally, I opted not to use regular expression for finding problematic words. Instead, I found problematic words by matching them individually, and then made corrections. As I proceeded further in the project, I thought that using regular expression would have allowed me to ultimately proceed faster and make corrections at a higher level than individual words. If I were to redo this project, I will use regular expressions instead.